<a href="https://colab.research.google.com/github/anruki/Procesamiento_YOLO/blob/main/p3_Ana_memoria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# PROCESAMIENTO DE VÍDEO PARA EL CUIDADO DE CABALLOS
---
### 🤓 **Autores**: Ana Robledano y Miguel Egido
### 📸 **Asignatura**: Procesamiento Multimedia
### 🧑 **Profesor**: Daniel Eusebio
### 👀 **Algoritmo**: You Only Look Once (YOLO)

---





# 1. INTRODUCCIÓN

En este proyecto se estudian casos de uso del procesamiento de vídeo junto con inteligencia artificial en el mundo ecuestre.

Se realizan las siguientes técnicas:
* __Preprocesamiento__: se aplican filtros, operaciones morfológicas,...
* __Inteligencia Artificial__: YOLO para la detección de caballos
* __Preprocesamiento__: para extraer características de los caballos detectados (su posición y velocidad).

El preprocesamiento facilita la tarea de reconocimiento de la IA y reduce la capacidad de computación necesaria para la ejecución del algoritmo YOLO.

El papel del procesamiento multimedia en este proyecto es crear un ``pipeline`` que transforma el vídeo input para que pueda ser procesado por el algoritmo YOLO.



# 1.1. Preprocesamiento

## 1.2.Algoritmo You Only Look One (YOLO)


El algoritmo You Only Look Once (YOLO), es un sistema de código abierto para detección de objetos en tiempo real, el cual hace uso de una única red neuronal convolucional para detectar objetos en imágenes. Para su funcionamiento, la red neuronal divide la imagen en regiones, prediciendo cuadros de identificación y probabilidades por cada región; las cajas son ponderadas a partir de las probabilidades predichas. El algoritmo aprende representaciones generalizables de los objetos, permitiendo un bajo error de detección para entradas nuevas, diferentes al conjunto de datos de entrenamiento.1

El algoritmo base corre a 45 cuadros por segundo (FPS) sin procesamiento de lote en un GPU Titan X. Debido a sus características de procesamiento, el algoritmo es utilizado en aplicaciones de detección de objetos en transmisión de video con retazo de señal menor a 25 milisegundos.2

Arquitectura
El modelo se implementó como una red neuronal convolucional y fue evaluado en el set de datos para detección de PASCAL VOC. Las capas convolucionales iniciales de la red se encargan de la extracción de características de la imagen, mientras que las capas de conexión completa predicen la probabilidad de salida y las coordenadas del objeto.

La red tiene 24 capas convolucionales seguidas por 2 capas de conexión completa; esta hace uso de capas de reducción de 1x1 seguidas capas convolucionales de 3x3. El modelo Fast YOLO hace uso de una red neuronal de 9 capas. La salida final del modelo tensor de predicción de 7x7x30.3​


Limitaciones
El algoritmo delimita fuertes restricciones espaciales en los límites de la caja de predicción dado que cada celda predice únicamente dos cajas y una clase; esto limita el número de objetos que se pueden detectar, lo cual hace que el algoritmo se vea limitado en la detección de objetos presentados en grupos.

Presenta problemas en la generalización de objetos visualizados en distintas configuraciones, al igual que en el proceso de post procesamiento de errores, tratando todos los errores con el mismo nivel de importancia.4​

### ¿Por qué Inteligencia Artificial?
Aunque el uso de Inteligencia Artificial supone un coste adicional en el procesamiento multimedia, aporta mayor flexibilidad de escenarios.

La detección de caballos podría realizarse manualmente mediante funciones de clusterización que no utilizan IA. No obstante, este proyecto está enfocado a facilitar la vida de ganaderos de distintos tipos y tamaños de animales. El modelo YOLO preentrenado permite la clasificación efectiva de distintas clases de animales. Concretamente para este

In [ ]:
pip install opencv-python ultralytics

In [ ]:
from ultralytics import YOLO

## 1.3. Aplicación real

A continuación se muestra el procesamiento de vídeo de un caballo para su detección mediante los apartados definidos anteriormente.

### Importación de librerías y del vídeo

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
video_path = 'sunday_vid.mov'
cap = cv2.VideoCapture(video_path)

### Carga del modelo `` YOLO V8 ``



In [ ]:
# load yolov8 model
model = YOLO('yolov8n.pt')

### Procesamiento del vídeo
Mediante un bucle que recorre cada frame

In [ ]:

ret = True
# read frames
while ret:
    ret, frame = cap.read()

    if ret:

        # detect objects
        # track objects
        results = model.track(frame, persist=True)

        # plot results
        # cv2.rectangle
        # cv2.putText
        frame_ = results[0].plot()

        # visualize
        cv2_imshow(frame_)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

In [ ]:
# Cargar modelo YOLOv8
model = YOLO('yolov8n.pt')

# Ruta del video de entrada
video_path = 'caballo.mp4'

# Cargar el video
cap = cv2.VideoCapture(video_path)

# CREAR VIDEO PROCESADO (OUTPUT)
#------------------------------------------------------------------
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = 'caballo_procesado_caballos.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

ret = True
#------------------------------------------------------------------

# PROCESAMIENTO
while ret:
    ret, frame = cap.read()
    if ret:
        # TRACKLING
        results = model.track(frame, persist=True)

        # DETECTAR SOLO CABALLOS
        for result in results[0].boxes:
            label = model.names[int(result.cls)]
            if label == 'horse':
                x1, y1, x2, y2 = map(int, result.xyxy[0])

                # PINTAR DE COLOR VERDE SI EL CABALLO SE ENCUENTRA DENTRO DEL RECINTO Y DE ROJO SI ESTA FUERA
                center_x = (x1 + x2) // 2
                color = (0, 255, 0) if center_x < width // 2 else (0, 0, 255)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        out.write(frame)

cap.release()
out.release()





0: 384x640 1 truck, 1 horse, 1 cow, 189.9ms
Speed: 6.5ms preprocess, 189.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 1 horse, 1 cow, 151.0ms
Speed: 3.4ms preprocess, 151.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 1 horse, 1 umbrella, 152.9ms
Speed: 3.3ms preprocess, 152.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 1 horse, 1 umbrella, 146.0ms
Speed: 4.9ms preprocess, 146.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 1 horse, 1 umbrella, 159.9ms
Speed: 3.4ms preprocess, 159.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 1 horse, 1 umbrella, 160.4ms
Speed: 4.5ms preprocess, 160.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 1 horse, 1 umbrella, 151.2ms
Speed: 3.7ms preprocess, 151.2ms inference, 1.2ms postprocess per image 